In [1]:
# import packages
import os
import dataclasses
from dataclasses import dataclass
import pandas as pd
import numpy as np
from datetime import date
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.parquet as pq
import itertools


In [ ]:
@dataclass
class edu:
    university_raw: str = "none"
    university_name: str = "none"
    education_number: np.int32 = np.int32(0)
    startdate: pa.date32 = date(2077, 9, 10)
    enddate: pa.date32 = date(2077, 9, 10)
    degree: str = "none"
    field: str = "none"
    degree_raw: str = "none"
    field_raw: str = "none"
    world_rank: np.double = np.double(0)
    us_rank: np.double = np.double(0)
    university_country: str = "none"
    university_location: str = "none"


edu_test = edu()

In [ ]:
dataclasses.asdict(edu_test).keys()
setattr(edu_test, "field", "Physics")
edu_test

In [3]:
# set environment folders
data_path = "../data"

In [ ]:

os.listdir("{data_path}/US_EDUC".format(data_path=data_path))

In [ ]:
# read user id dataset
ids = pq.read_table("{data_path}/unique_user_id_US_EDUC.parquet".format(data_path=data_path))
# sort user id: later would useful in filtering
ids.sort_by("user_id")
# min_id = ids.column("user_id")[0]
# max_id = ids.column("user_id")[-1]


In [ ]:
ids.column("user_id")

In [ ]:
# check min and max id
print("min id is:", min_id)
print("max id is:", max_id)

In [4]:
# read education
edus = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_education.parquet".format(data_path=data_path))
# display edus metadata
edus.metadata

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 14
  num_rows: 62129675
  num_row_groups: 60
  format_version: 2.6
  serialized_size: 105949

In [ ]:
# try reading one rowgroup and check memory usage
group0 = edus.read_row_group(0)
group0.slice(length=6)

expr = pc.field("user_id") == 617479683
group0.filter(expr).sort_by("enddate")

In [ ]:
group0.column("user_id").unique()

In [ ]:
# check group0 properties
# group0.slice(length=10).sort_by([("user_id","ascending"),("enddate","ascending")])
# group0.filter(group0.field("user_id") == 323)
group0.field("user_id")
expr = pc.field("user_id") < 323
sub0 = group0.filter(mask = expr, null_selection_behavior = "drop")
sub0.num_rows

In [ ]:
print(group0.column("user_id").index(111))
print(group0.column("user_id").index(130452445))

In [ ]:
id_expr = pc.field("user_id") == 130452445
ids.filter(id_expr)

In [ ]:
# check user dataset
edus = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_part_1_0_249.parquet".format(data_path=data_path))
edu0 = edus.read_row_group(0)
edu0.schema

In [ ]:
edu0.slice(length=5).column("user_id")

In [ ]:
# check user_position dataset
pos = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_position_part_1_00_0_249.parquet".format(data_path=data_path))
pos0 = pos.read_row_group(0)
pos0.schema

NameError: name 'data_path' is not defined

In [ ]:
print(pos0.slice(length=1).column(0)[0])
print(pos0.column_names[0])
print(pos0.column("user_id")[0])

In [ ]:
# pos0.select(["title_raw", "role_k1500", "job_category", "role_k50", "role_k150"])
for (col_name, col_type) in itertools.zip_longest(pos0.schema.names, pos0.schema.types):
    print(col_name,":", col_type)

In [ ]:
# check user_skill dataset
skill = pq.ParquetFile("{data_path}/US_EDUC/US_EDUC_user_skill_part_0000_0049.parquet".format(data_path=data_path))
skill0 = skill.read_row_group(0)
skill0.num_columns
skill0.column("user_id")[0]

In [ ]:
skill0.slice(length=3)

In [ ]:
df = pd.DataFrame({"a":[1,2,3], "b":[2,3,4], "c":pd.Series([pd.NA] * 3)})
print(df)
df.loc[df["a"]==2, "c"] = 10
print(df)

In [ ]:
for i in range(3,-1,-1):
    print(i)